In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /home/adityasidharta/git/shopee_data_science
%env PROJECT_PATH /home/adityasidharta/git/shopee_data_science

In [ ]:
from utils.envs import *
from model.heuristic.mobile.extractor import *
import pandas as pd
from model.heuristic.mobile.enricher import *

In [ ]:
Ext = Extractor()
Enr = Enricher()

In [ ]:
import json

def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

In [ ]:
test_df = pd.read_csv(mobile_test_repo)
mobile_json = load_json(mobile_profile_json)

In [ ]:
mobile_columns = ['Operating System', 'Features',
               'Network Connections', 'Memory RAM', 'Brand', 'Warranty Period',
               'Storage Capacity', 'Color Family', 'Phone Model', 'Camera',
               'Phone Screen Size']

In [ ]:
def create_pred_dataframe(extracted, item_id):
    result_list = []
    for feature in mobile_columns:
        extracted_value = extracted[feature]
        tagging = ''
        if isinstance(extracted_value, list):
            if len(extracted_value) == 1:
                extracted_value = extracted_value[0]
                if extracted_value in mobile_json[feature].keys():
                    tagging = mobile_json[feature][extracted_value]
        else:
            if extracted_value in mobile_json[feature].keys():
                tagging = mobile_json[feature][extracted_value]
        result_list.append({
            'id': str(item_id) + '_{}'.format(feature),
            'tagging': tagging
        })
    return pd.DataFrame(result_list)

In [ ]:
from tqdm import tqdm

In [ ]:
len(test_df)

In [ ]:
enriched_list = []

Ext = Extractor()
Enr = Enricher()

for idx, rows in tqdm(test_df.iterrows()):
    extracted = Ext.extract_from_title(rows.title)
    enriched = Enr.enrich(extracted)
    
    enriched_df = create_pred_dataframe(enriched, rows.itemid)
    enriched_list.append(enriched_df)

In [ ]:
result_enriched_df = pd.concat(enriched_list)

In [ ]:
result_enriched_df.head()

In [ ]:
kaggle_df = pd.read_csv('/home/adityasidharta/git/shopee_data_science/output/result/result_20190314-225622.csv')

In [ ]:
merge_df = kaggle_df.merge(result_enriched_df, how='left', on='id')
merge_df = merge_df.fillna('')

In [ ]:
def final_tagging(rows):
    first_prediction = str(rows.tagging_x.split(' ')[0])
    second_prediction = str(rows.tagging_x.split(' ')[1])
    kyle_prediction = str(rows.tagging_y)
    if kyle_prediction == '':
        return rows.tagging_x
    else:
        if kyle_prediction != first_prediction:
            return "{} {}".format(kyle_prediction, first_prediction)
        else:
            return "{} {}".format(kyle_prediction, second_prediction)

In [ ]:
merge_df['tagging'] = merge_df.apply(final_tagging, axis = 1)

In [ ]:
merge_df[merge_df.tagging_y != ''].head(20)

In [ ]:
merge_df.shape

In [ ]:
merge_df = merge_df[['id', 'tagging']]

In [ ]:
merge_df.to_csv(os.path.join(output_path, 'result/prediction_kyle_mobile_19_march.csv'), index=False)

In [ ]:
merge_df